---

# 13.2.3 TPU Training

TPUs are available in Colab. Here is a template to get you started.

#### Note

Sadly, given the huge popularity of the service, Google has introduced paid tiers and reduce (eliminated?) free TPU access. I do remember being able to experiment with this, but of late I have not been able to connect to a TPU runtime any longer... It might be that the free version of Colab no longer offers TPUs, even if the option is there. Let me know if you are able to connect!

In [ ]:
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

# connect to the TPU cluster
# https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/guide/tpu.ipynb#scrollTo=7SO23K8oRpjI
try:
    resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
    tf.config.experimental_connect_to_cluster(resolver)
    # This is the TPU initialization code that has to be at the beginning.
    tf.tpu.experimental.initialize_tpu_system(resolver)
    print("All devices: ", tf.config.list_logical_devices('TPU'))
    strategy = tf.distribute.TPUStrategy(resolver)
    print(f"Number of replicas: {strategy.num_replicas_in_sync}") # a TPU has 8 cores 

except ValueError:
    strategy = tf.distribute.get_strategy() 
    print("No TPU found, falling back onto default strategy.")

(Listing 13.4)

In [ ]:
# functional encapsulation: everything inside the function
# will be created following the strategy
def build_model(input_size):
    inputs = tf.keras.Input((input_size, input_size, 3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = tf.keras.applications.resnet.ResNet50(
        weights=None, include_top=False, pooling="max"
    )(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer="rmsprop",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

# ↓ we create our model in the 'scope' of the strategy
with strategy.scope():
    model = build_model(input_size=32)
# ↑ --------------------------------------------------

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

In [ ]:
# the first epoch is always much slower, as the system compiles every single step
# once that compilation has taken place, the speedup gain is great
model.fit(
    x_train,
    y_train, 
    epochs=10,
    batch_size=1024 # notice the large batch size, TPUs have a large capacity!
)

---

TensorFlow has various Colabs available showing use cases of TPUs:
- See [this page](https://www.tensorflow.org/guide/tpu), which has more links at the bottom.